In [1]:
# point to root dir
import os

os.chdir("../")
os.getcwd()

'c:\\Users\\HP\\Desktop\\GenHack\\Genhack_competition'

In [2]:
# imports
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [3]:
# load data
# validation data
data_val = pd.read_csv(
    "Data/validation_samples.csv", 
    header=None, 
    index_col=0
)

# generated
# FinPred
data_fin = pd.read_csv(
    "Data/generated_samples_Fin.csv", 
    header=None
)

# Zhow A
data_zhow = pd.read_csv(
    "Data/generated_samples_Zhow.csv", 
    index_col=0
)
# rename cols
data_zhow.rename(
    columns={ f"I{i + 1}": i for i in range(4) }, 
    inplace=True
)


In [4]:
# visualize validation

# data
data = data_val

fig = go.Figure(
    data=[
        go.Scatter(
            y=data[index],
            mode="lines",
            name=index
        )
    for index in data.columns]
)

fig.update_layout(
    title="Indexes",
    yaxis_range=[-0.01, 0.1]
)

fig.show()

In [5]:
# compute metrics
from py_scripts.marginals import Anderson_Darling
from py_scripts.dependance_absolute_kendall_error import Absolute_Kendall_error


real_anderson_fin = Anderson_Darling(
    data_fin.values,
    data_val.values
)

real_kendall_fin = Absolute_Kendall_error(
    data_fin.values,
    data_val.values
)

In [6]:
real_anderson_zhow = Anderson_Darling(
    data_zhow.values,
    data_val.values
)

real_kendall_zhow = Absolute_Kendall_error(
    data_zhow.values,
    data_val.values
)

In [7]:
print(real_anderson_fin.round(2), real_kendall_fin.round(3))
print(real_anderson_zhow.round(2), real_kendall_zhow.round(3))

80.65 0.069
9.74 0.145


In [8]:
# load leader board
from py_scripts.utils import load_dict


leader_board = load_dict("reverse_scores/leaderboard")

In [9]:
# standard deviations
z_anderson_fin = leader_board["FinPred"]["anderson"]["z-score"]
z_anderson_zhow = leader_board["Zhow A311"]["anderson"]["z-score"]

z_kendall_fin = leader_board["FinPred"]["kendall"]["z-score"]
z_kendall_zhow = leader_board["Zhow A311"]["kendall"]["z-score"]

sigma_anderson = (real_anderson_fin - real_anderson_zhow) / (z_anderson_fin - z_anderson_zhow)
sigma_kendall = (real_kendall_fin - real_kendall_zhow) / (z_kendall_fin - z_kendall_zhow)

print(sigma_anderson)
print(sigma_kendall)

5311.648547872344
0.15760861634970177


In [10]:
# mean
mean_anderson = real_anderson_fin - z_anderson_fin * sigma_anderson
mean_kendall = real_kendall_fin - z_kendall_fin * sigma_kendall

print(mean_anderson)
print(mean_kendall)

-575.4248625362086
0.08228086420363


In [11]:
dict_metrics_stats = {
    "anderson": {
        "sigma": sigma_anderson,
        "mean": mean_anderson
    },
    "kendall": {
        "sigma": sigma_kendall,
        "mean": mean_kendall
    }
}

In [12]:
# fill other scores
# loop over teams
for team_name, team_scores in leader_board.items():
    # loop over scores
    for metric_name, metric_scores in team_scores.items():
        z_score = metric_scores["z-score"]

        leader_board[team_name][metric_name]["real"] = dict_metrics_stats[metric_name]["sigma"] * z_score + dict_metrics_stats[metric_name]["mean"]

In [13]:
leader_board

{'Winx': {'anderson': {'board': 647.5409999999999,
   'z-score': 0.37869010787150503,
   'real': 1436.0438990330927},
  'kendall': {'board': 747.761,
   'z-score': 0.6674605507048509,
   'real': 0.18747839806823152}},
 '5 Layers of Stonks': {'anderson': {'board': 647.3109999999999,
   'z-score': 0.37807079924354703,
   'real': 1432.7543492587147},
  'kendall': {'board': 744.1809999999999,
   'z-score': 0.6562893012562411,
   'real': 0.18571771289973876}},
 'DL Manjikai': {'anderson': {'board': 647.261,
   'z-score': 0.377936186127173,
   'real': 1432.039331694602},
  'kendall': {'board': 742.081,
   'z-score': 0.6497743340973121,
   'real': 0.1846908979402562}},
 'Terran': {'anderson': {'board': 647.031,
   'z-score': 0.3773170539471422,
   'real': 1428.7507191496004},
  'kendall': {'board': 755.011,
   'z-score': 0.6903438155675782,
   'real': 0.1910849977808097}},
 'Lbitrik': {'anderson': {'board': 646.761,
   'z-score': 0.376590431092823,
   'real': 1424.8911539206047},
  'kendall':

In [14]:
# from py_scripts.utils import save_dict

# save_dict(
#     leader_board,
#     "reverse_scores/leaderboard_completed"
# )